In [ ]:
%load_ext autoreload
%autoreload 2

# Buffer

> ABC Buffer

In [ ]:
#| default_exp storage.buffer.buffer

In [ ]:
#| export
from __future__ import annotations
import abc
import weakref
from dataclasses import dataclass
from typing import ClassVar, Generic, Optional, Tuple, get_args
import numpy as np
import pandas as pd  # type: ignore

In [ ]:
#| export
from data_io_nbdev.storage.pool.pool import Pool
from data_io_nbdev.data.core import ItemT, PoolQuery

In [ ]:
#| export
@dataclass(kw_only=True)
class Buffer(abc.ABC, Generic[ItemT]):
    """
    The abstract class for providing an buffer interface for data processing.

    Buffer is the internal dynamic memory object for pooling the experience tuples.
    It provides the following interface for inheriting classes (MongoBuffer, DaskBuffer, etc.):

    Methods:
        - load()
        - save()
        - store()
        - sample()


    Attributes:
        - pool: the pool object for storing the data
        - batch_size: the batch size for sampling
        - _type_T: the type of the data item (e.g. Record, Episode, etc.)
    """

    pool: Optional[Pool]
    batch_size: int
    _type_T: ClassVar[str]

    def __init_subclass__(cls):
        """Get the concrete type of the data item (e.g. Record, Episode, etc.)"""
        cls._type_T = get_args(cls.__orig_bases__[0])[0].__name__  # type: ignore
        # print(
        #     f"Pool.__init_subclass__(): {cls._type_T} in {cls.__name__} from {cls.__base__.__name__}"
        # )

    def __post_init__(self):
        """User weakref finalizer to make sure close is called when the object is destroyed"""
        self._finalizer = weakref.finalize(self, self.close)

    @abc.abstractmethod
    def load(self):
        """
        load buffer from pool
        """
        pass

    @abc.abstractmethod
    def close(self):
        """
        close the pool, for destructor
        """
        pass

    # @abc.abstractmethod
    def store(self, episode: ItemT):
        """
        Deposit an item (record/episode) into the pool
        """
        self.pool.store(episode)

    # @abc.abstractmethod
    def find(self, query: PoolQuery):
        """
        find an itme by id or name.
        """
        return self.pool.find(query)

    @abc.abstractmethod
    def sample(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        sample data pool to get (state, action, reward, nstate) as a tuple of 4 DataFrames
        """

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
show_doc(Buffer.store)

---

### Buffer.store

>      Buffer.store (episode:~ItemT)

Deposit an item (record/episode) into the pool

In [ ]:
show_doc(Buffer.find)

---

### Buffer.find

>      Buffer.find (query:data_io_nbdev.data.core.PoolQuery)

find an itme by id or name.

In [ ]:
show_doc(Buffer.sample)

---

### Buffer.sample

>      Buffer.sample ()

sample data pool to get (state, action, reward, nstate) as a tuple of 4 DataFrames

In [ ]:
show_doc(Buffer.load)

---

### Buffer.load

>      Buffer.load ()

load buffer from pool

In [ ]:
show_doc(Buffer.close)

---

### Buffer.close

>      Buffer.close ()

close the pool, for destructor

In [ ]:
show_doc(Buffer.__init_subclass__)

---

### Buffer.__init_subclass__

>      Buffer.__init_subclass__ ()

Get the concrete type of the data item (e.g. Record, Episode, etc.)

In [ ]:
show_doc(Buffer.__post_init__)

---

### Buffer.__post_init__

>      Buffer.__post_init__ ()

User weakref finalizer to make sure close is called when the object is destroyed

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()